In [2]:
import pandas as pd

bureau = "https://raw.githubusercontent.com/JY6299/CR_Modeling_2/refs/heads/main/bureau_data.csv"
loan = "https://raw.githubusercontent.com/JY6299/CR_Modeling_2/refs/heads/main/loans.csv"
cust = "https://raw.githubusercontent.com/JY6299/CR_Modeling_2/refs/heads/main/customers.csv"

bureau_data = pd.read_csv(bureau)
loan_data = pd.read_csv(loan)
cust_data = pd.read_csv(cust)

bureau_data.shape, loan_data.shape, cust_data.shape

((50000, 8), (50000, 15), (50000, 12))

Successfully Implemented Data.
In this Project, aim to build several ML models to compare performance, and select one as final model for prediction.
Built a roburst credit risk classification model for PD
PipeLine: 1.Data Preprocessing
          2.EDA and feature selection
          3.SMOTE to resample response variable(PD)
          4.Model Selection
          5.Model Hyperparameter Tuning with Optuna
          6.Model Performance Monitoring

In [6]:
bureau_data.isnull().sum(), loan_data.isnull().sum()
cust_data.isnull()

,cust_id,age,gender,marital_status,employment_status,income,number_of_dependants,residence_type,years_at_current_address,city,state,zipcode
0,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
49995,False,False,False,False,False,False,False,False,False,False,False,False
49996,False,False,False,False,False,False,False,False,False,False,False,False
49997,False,False,False,False,False,False,False,False,False,False,False,False
49998,False,False,False,False,False,False,False,False,False,False,False,False
